# Import Modules

In [13]:
import numpy as np
import random
import scipy.io as scio
import torch
import torch.nn as nn

from models.cgan import Generator, Discriminator

# CGAN

A conditional generative adversarial network (CGAN) is a type of GAN that also takes advantage of labels during the training process.  

Generator — Given a label and random array as input, this network generates data with the same structure as the training data observations corresponding to the same label.  

Discriminator — Given batches of labeled data containing observations from both the training data and generated data from the generator, this network attempts to classify the observations as "real" or "generated".  

![Gan](https://www.mathworks.com/help/examples/nnet/win64/TrainConditionalGenerativeAdversarialNetworkCGANExample_02.png)

## GAN Loss

- Loss_D - discriminator loss calculated as the sum of losses for the all real and all fake batches : WANT Maximize
\begin{equation}
(log(D(x)) + log(1 - D(G(z)))
\end{equation}
- Loss_G - generator loss calculated as log(D(G(z))) () : WANT Maximize
\begin{equation}
log(D(G(z)))
\end{equation}
- D(x) - the average output (across the batch) of the discriminator for the all real batch. This should start close to 1 then theoretically converge to 0.5 when G gets better. Think about why this is.  
- D(G(z)) - average discriminator outputs for the all fake batch. The first number is before D is updated and the second number is after D is updated. These numbers should start near 0 and converge to 0.5 as G gets better.  

*BCELoss
\begin{equation}
l_{n}(x_{n},y_{n})=-\left[y_{n} \cdot \log x_{n}+\left(1-y_{n}\right) \cdot \log \left(1-x_{n}\right)\right]
\end{equation}

### Loss in Train  
Fake Label : 0  & True Label : 1  
Fake input : G(z) & Real input : x  
\begin{equation}
\text{d_loss_fake} = -log(1-D(G(z)) \quad \& \quad
\text{d_loss_true} = -log(D(x))  \\
\text{d_loss} = -log(1-D(G(z)) + -log(D(x)) \\
\text{g_loss} = -log(D(G(z))  \\
\end{equation}


### General Behavior

In general, a GAN has converged when both the generator and discrimimator losses have both converged, each around a stable value. Visually, the loss curve of a GAN that has trained properly looks something like this:  
![image](https://nchlis.github.io/2019_11_22/loss_pix2pix_RF1.0_disRF1.0_batchSize1_earlyStoppingFalse.png)

In the plot above we can see that the as the discriminator gets better, the loss of the generator increases as a result, until they both stabilize and the entire GAN has practically converged after 100 epochs or so. There is a small spike in generator loss at approximately 175 epochs but it returns to normal shortly after, so its probably nothing to worry about. While the above is an indication that the network as trained nicely, the only true way to validate this, is to evaluate it by translating and plotting some images of the test set.

# Define Train Function

In [14]:
# train
# Loss of original GAN paper.
def train(data, num_epochs=10, batch_size=32, noise_size = 64, lr=0.0002, device='cpu', class_num=10, tag=""):
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    
    # original was MSE
    adversarial_criterion = nn.BCELoss()
    discriminator = Discriminator(num_classes=class_num).to(device)
    generator = Generator(num_classes=class_num).to(device)
    
    fixed_noise = torch.randn([batch_size, noise_size])
    fixed_conditional = torch.randint(0, class_num, (batch_size,))
    
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    x = data[0]
    y = data[1]
    global G_Loss, D_Loss, D_real, D_fake, sr_list
    G_Loss = []
    D_Loss = []
    sr_list = []
    D_real = []
    D_fake = []
    
    index = 0
    for epoch in np.arange(num_epochs):
        discriminator.train()
        generator.train()
        
        
        for i, (inputs, target) in enumerate(zip(x,y)):
            inputs = torch.Tensor(inputs).to(device)
            target = torch.IntTensor(target).to(device)
            
            real_label = torch.ones(batch_size, 1).to(device)
            fake_label = torch.zeros(batch_size, 1).to(device)
            
            noise = torch.randn([batch_size, noise_size]).to(device)
            conditional = torch.randint(0, class_num, (batch_size,)).to(device)
            
            # d_loss true
            real_output = discriminator(inputs, target)
            d_loss_real = adversarial_criterion(real_output, real_label)
            D_x = real_output.detach().cpu().mean().item()
            
            # d_loss fake
            fake = generator(noise, conditional)
            # Only update loss backward through discriminator network. Use detach
            fake_output = discriminator(fake.detach(), conditional)
            d_loss_fake = adversarial_criterion(fake_output, fake_label)
            D_G_z = fake_output.detach().cpu().mean().item()
            
            # d_loss total
            d_loss = d_loss_fake + d_loss_real
            
            # d train
            discriminator.zero_grad()
            generator.zero_grad()
            d_loss.backward()
            discriminator_optimizer.step()
            
            # g loss
            # Since we just updated D, perform another forward pass of all-fake batch through D
            fake = generator(noise, conditional)
            fake_output = discriminator(fake, conditional)
            g_loss = adversarial_criterion(fake_output, real_label)
            
            # g train
            discriminator.zero_grad()
            generator.zero_grad()
            g_loss.backward()
            discriminator_optimizer.step()
            
            errG = g_loss.detach().cpu().mean().item()
            errD = d_loss.detach().cpu().mean().item()
            
            G_Loss.append(errG)
            D_Loss.append(errD)
            D_real.append(D_x)
            D_fake.append(D_G_z)
            
            if(index%100 == 0):
                print("Epoch:", epoch, "Global Iter: ",index,"Current G Loss : %.2f" % errG,"Current D Loss : %.2f" % errD, end=" ")
                print("D(x): %.2f" %  D_x, "D(G(z)): %.2f" % D_G_z)
            index+=1

        # Evaluation
        with torch.no_grad():
            generator.eval()
            sr = generator(fixed_noise.to(device), fixed_conditional.to(device))
            ## make some visualization or saving file func
            # Func(sr, path)
            sr_list.append(sr.detach().cpu())
            
            torch.save(generator.state_dict(), f"./model_ckpt/ckpt{epoch}_generator_{tag}.pt")
            torch.save(discriminator.state_dict(), f"./model_ckpt/ckpt{epoch}_discriminator_{tag}.pt")   

# Load Dataset & Data Batch Split

In [15]:
cgan_dataset = scio.loadmat("./gan_dataset/cgan_dataset.mat")

In [16]:
batch_size = 32
x = []
y = []
iteration = cgan_dataset["X"].shape[0] // batch_size
for i in range((iteration-1)):
    x.append(cgan_dataset['X'][batch_size*i: batch_size*(i+1)])
    y.append(cgan_dataset['Y'][0][batch_size*i: batch_size*(i+1)])
    
x = np.array(x, dtype=float)
x = np.expand_dims(x, 2)
y = np.array(y, dtype=int)

x.shape, y.shape

((695, 32, 1, 200), (695, 32))

# TRAIN

In [17]:
train(data=[x,y], num_epochs=100, batch_size=32, lr=0.00002, device='cuda', class_num=18, tag="")

Epoch: 0 Global Iter:  0 Current G Loss : 0.69 Current D Loss : 1.39 D(x): 0.50 D(G(z)): 0.50
Epoch: 0 Global Iter:  100 Current G Loss : 0.68 Current D Loss : 1.33 D(x): 0.53 D(G(z)): 0.51
Epoch: 0 Global Iter:  200 Current G Loss : 0.61 Current D Loss : 1.15 D(x): 0.71 D(G(z)): 0.55
Epoch: 0 Global Iter:  300 Current G Loss : 0.51 Current D Loss : 1.20 D(x): 0.79 D(G(z)): 0.60
Epoch: 0 Global Iter:  400 Current G Loss : 0.49 Current D Loss : 1.15 D(x): 0.86 D(G(z)): 0.62
Epoch: 0 Global Iter:  500 Current G Loss : 0.52 Current D Loss : 1.25 D(x): 0.74 D(G(z)): 0.60
Epoch: 0 Global Iter:  600 Current G Loss : 0.55 Current D Loss : 1.07 D(x): 0.87 D(G(z)): 0.59
Epoch: 1 Global Iter:  700 Current G Loss : 0.53 Current D Loss : 1.06 D(x): 0.91 D(G(z)): 0.60
Epoch: 1 Global Iter:  800 Current G Loss : 0.57 Current D Loss : 1.06 D(x): 0.86 D(G(z)): 0.58
Epoch: 1 Global Iter:  900 Current G Loss : 0.58 Current D Loss : 0.94 D(x): 0.94 D(G(z)): 0.57
Epoch: 1 Global Iter:  1000 Current G Loss

## Save Train Results

In [18]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [19]:
import datetime

In [20]:
time_tag = datetime.datetime.now().date().strftime("%m%d")

In [21]:
def make_average_epoch(data, iter_in_epoch):
    arr = np.array(data)
    return np.nanmean(np.pad(arr.astype(float), (0, iter_in_epoch - arr.size%iter_in_epoch), mode='constant', constant_values=np.NaN).reshape(-1, iter_in_epoch), axis=1)

In [22]:
iter_in_epoch = x.shape[0]
avg_results = pd.DataFrame(np.array(list(map(lambda x : make_average_epoch(x,iter_in_epoch), [G_Loss, D_Loss, D_real, D_fake]))).T)
avg_results.columns = ["G_Loss", "D_Loss", "D_real", "D_fake"]
data = [go.Scatter(x=avg_results.index, y=avg_results.G_Loss, name="G_Loss"),
        go.Scatter(x=avg_results.index, y=avg_results.D_Loss, name="D_Loss"),
        go.Scatter(x=avg_results.index, y=avg_results.D_real, name="D_real"),
        go.Scatter(x=avg_results.index, y=avg_results.D_fake, name="D_fake")]
fig = go.Figure(data=data)
fig.write_html(f"./gan_loss/cgan_results_averaged_{time_tag}.html")

C:\Users\user\anaconda3\envs\cuda37\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning:

Mean of empty slice



---

In [27]:
torch.cuda.empty_cache()

# Load Model and Make Dataset

In [23]:
class_num = 18
batch_size = 100
noise_size = 64

model = Generator(num_classes=class_num)
_input = torch.randn([batch_size, noise_size])
_class =  torch.randint(0, class_num, (batch_size,))

### Load from state dict

In [24]:
########################################
i = 694
tag = ""
########################################

model.load_state_dict(torch.load(f"./model_ckpt/ckpt{i}_generator_{tag}.pt"))

<All keys matched successfully>

## Make dataset

In [25]:
results = model(_input, _class)

In [26]:
np.save("./gan_results/gan_output.npy", results.detach().numpy())